<a href="https://colab.research.google.com/github/tlkp1021/hello-git/blob/master/json_cleaning_kkday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python
# coding: utf-8
# KKDAY Activity json cleaning

# In[1]:

import pandas as pd
import pydata_google_auth
import pandas as pd
import numpy as np
#get_ipython().run_line_magic('matplotlib', 'inline')
#import matplotlib.pyplot as plt
#import seaborn as sns
import scipy.stats as stats
import json





In [33]:


# In[2]:


# KKDAY SQL

# kkday_cols = ["id", "name", "introduction", "rating_count", "rating_star", "order_count", "days" , 
#"hours","minutes" ,"duration","confirm_hour","instant_booking"]

#Read data

query = """
select * from
(SELECT spider_date , competitor_code , activity_id, language, currency  , source_data , row_number() over (partition by activity_id , language , currency  order by spider_date desc) date_order_desc
FROM `tech-scraping.competitor_price_scraping.full_site_source_data` 
WHERE spider_date >= "2020-06-01" 
and competitor_code = 'kkday')
where date_order_desc = 1
"""
project_id = 'klook-dm-bd-ops'

credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

tech_df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
tech_df.head(10)




,spider_date,competitor_code,activity_id,language,currency,source_data,date_order_desc
0,2020-06-01,kkday,10000,ja,JPY,"{""id"": 10000, ""prod_oid"": 10000, ""name"": ""\u30...",1
1,2020-06-01,kkday,10000,ko,KRW,"{""id"": 10000, ""prod_oid"": 10000, ""name"": ""\u30...",1
2,2020-06-01,kkday,100020,ja,JPY,"{""id"": 100020, ""prod_oid"": 100020, ""name"": ""\u...",1
3,2020-06-01,kkday,100023,zh-tw,TWD,"{""id"": 100023, ""prod_oid"": 100023, ""name"": ""\u...",1
4,2020-06-01,kkday,100024,zh-my,MYR,"{""id"": 100024, ""prod_oid"": 100024, ""name"": ""\u...",1
5,2020-06-01,kkday,100044,zh-cn,CNY,"{""id"": 100044, ""prod_oid"": 100044, ""name"": ""\u...",1
6,2020-06-01,kkday,100048,zh-tw,TWD,"{""id"": 100048, ""prod_oid"": 100048, ""name"": ""\u...",1
7,2020-06-01,kkday,10008,en-sg,SGD,"{""id"": 10008, ""prod_oid"": 10008, ""name"": ""Kore...",1
8,2020-06-01,kkday,100095,zh-cn,CNY,"{""id"": 100095, ""prod_oid"": 100095, ""name"": ""\u...",1
9,2020-06-01,kkday,100104,zh-cn,CNY,"{""id"": 100104, ""prod_oid"": 100104, ""name"": ""\u...",1


In [0]:

# In[3]:


#"id", "name", "introduction", "rating_count", "rating_star", "order_count" ,"duration","confirm_hour","instant_booking"
data_all = []
kkday_col = ["date","competitor_code","language","currency","id", "name", "act_desc", "review_count", "rating", "order_count" ,"duration",
             "confirmation_time","instant_booking","instant_use","free_refund_policy","city","country","earliest_sale_date",
             "raw_currency","price","sale_price"]




for i in tech_df.index:
    raw = json.loads(tech_df['source_data'][i])
    data = {'date': tech_df['spider_date'][i],
            'competitor_code': tech_df['competitor_code'][i],
            'language': tech_df['language'][i],
            'currency': tech_df['currency'][i],
            'id': raw['id'],
            'name': raw['name'],
            'act_desc': raw['introduction'],
            'review_count': raw['rating_count'],
            'rating': raw['rating_star'],
            'order_count': raw['order_count'],
            'duration': raw['duration'],
            'confirmation_time': raw['confirm_hour'],
            'instant_booking': raw['instant_booking'],
            'instant_use': raw['instant_use'],
            'free_refund_policy': raw['free_refund_policy'],         
            'city' : raw['show_city_name'],
            'country' : raw['show_country_name'],
            'earliest_sale_date' : raw['earliest_sale_date'],
            'raw_currency' : raw['currency'],
            'price' : raw['price'],
            'sale_price' : raw['sale_price']
            }
    data_all.append(data)    
df = pd.DataFrame(data_all, columns=kkday_col) 




In [62]:

def clean_html_tag (my_list):
  for i, element in enumerate(my_list):
    element = element.replace('<br>','')
    element = element.replace('</br>','')
    element = element.replace('<','')
    element = element.replace('/','')
    element = element.replace('\\','')
    element = element.replace('>','')
    element = element.replace('\r','')
    element = element.replace('\br','')
    element = element.replace('\n','')    
    my_list[i] = element
  return my_list

df.name = clean_html_tag(df.name)
df.act_desc = clean_html_tag(df.act_desc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [53]:
df.describe

<bound method NDFrame.describe of             date competitor_code language  ... raw_currency   price sale_price
0     2020-06-01           kkday       ja  ...          USD   21.80      21.80
1     2020-06-01           kkday       ko  ...          USD   21.80      21.80
2     2020-06-01           kkday       ja  ...          USD  255.09     255.09
3     2020-06-01           kkday    zh-tw  ...          USD  183.58     183.58
4     2020-06-01           kkday    zh-my  ...          USD   16.19      16.19
...          ...             ...      ...  ...          ...     ...        ...
75266 2020-06-01           kkday    zh-cn  ...          HKD  155.00     155.00
75267 2020-06-01           kkday    zh-tw  ...          USD   19.99      19.99
75268 2020-06-01           kkday    zh-hk  ...          USD   19.99      19.99
75269 2020-06-01           kkday    en-id  ...          USD   18.06      18.06
75270 2020-06-01           kkday       en  ...          USD   19.99      19.99

[75271 rows x 21 

In [63]:
#checking df
df.iloc[2345:2348]




,date,competitor_code,language,currency,id,name,act_desc,review_count,rating,order_count,duration,confirmation_time,instant_booking,instant_use,free_refund_policy,city,country,earliest_sale_date,raw_currency,price,sale_price
2345,2020-06-01,kkday,en-sg,SGD,24798,Grand Canyon West Tour With Skywalk Ticket,• Convenient pick-up at your downtown hotel in...,1,5.0,3,600,0.0,True,True,False,"Arizona, Multiple cities",United States,20200601,KRW,187198.0,187198.0
2346,2020-06-01,kkday,en-id,IDR,24799,GC- South Rim VIP Small Group Tour,VIP Small group luxury tourbr Hotel pick-ups b...,1,5.0,3,840,0.0,True,False,False,Las Vegas,United States,20200605,USD,287.0,287.0
2347,2020-06-01,kkday,zh-hk,HKD,24801,Vegas City Lights Helicopter Tour,Check-in: 30 Minutes Prior To Flight Time.\nFl...,0,0.0,0,0,48.0,False,False,False,拉斯維加斯,美國,None,USD,109.0,109.0


In [64]:
# In[6]:
#upload to BQ

table_id = 'temp.competitor_act_attribute_kkday'
df.to_gbq(table_id, 
                 project_id,
                 if_exists='replace'
                 )
















0it [00:00, ?it/s]












1it [00:20, 20.84s/it]
